**Tabalho Prático 1 – Análise de Crédito**

**Alunos:

GIOVANNI SURIANI FERREIRA

JORGE ALIOMAR TROCOLI ABDON DANTAS**

**Enunciado(s):**

Cada dupla deve escolher um problema no Kaggle. Elaborar uma solução e apresentar para turma no dia da entrega. A solução da dupla deve ser entregue no Moodle em um Python Notebook. Especificamente, a dupla deve:
- Pesquisar um problema interessante no Kaggle que envolva dados tabulares/estruturados;
- Realizar uma análise exploratória dos dados da competição. Isto é, visualizar os dados, tentar identificar dados faltantes, instâncias ruidosas e tendências nos dados;
- Entender e implementar a métrica de avaliação da competição;
- Implementar uma solução simples para o problema da competição para gerar um baseline (isto é, o desempenho mínimo que o sua abordagem deve alcançar);
- Estudar soluções para problemas semelhantes ao da competição escolhida. A dupla pode pesquisar, por exemplo, em artigos acadêmicos, artigos técnicos da Internet e o no próprio Kaggle;
- Implementar diferentes abordagens para a competição e reportar o desempenho de cada abordagem implementada;
- Realizar uma avaliação de parâmetro (hyperparameter tuning) das abordagens escolhidas para o problema. Para realizar a avaliação dos parâmetros você pode utilizar GridSearchCV ou Optuna.
- Além de outras abordagens, a dupla deve:
    - Propor pelo menos uma abordagem que envolva meta-learning. Ou seja, a dupla deve propor uma forma diferente de combinar vários modelos. Por exemplo, utilize a abordagem do Random Forest, porém com vários classificadores ou invés de utilizar apenas Árvores de Decisão.
- Para cada abordagem apresentada, a dupla deve apresentar:
    - desempenho no conjunto de treino, o desempenho deve ser medido com uma validação cruzada com 10 folds (10-fold cross-validation).
    - desempenho no conjunto de teste reportado pelo Kaggle

# DESAFIO E PROBLEMA ESCOLHIDO
Home Credit Default Risk (https://www.kaggle.com/competitions/home-credit-default-risk)

O desafio consiste em melhorar os modelos preditivos, da HOME CREDIT, de crédito voltados a pessoas sem histórico bancário. Utilizando dados alternativos como registros de telecomunicações e transações, a empresa quer garantir que mais pessoas aptas recebam empréstimos justos e sustentáveis. O desafio propõe que os participantes usem técnicas de ciência de dados e machine learning para prever com mais precisão a capacidade de pagamento dos clientes.

O conjunto de dados contém informações detalhadas sobre o histórico financeiro de clientes da Home Credit e de outras instituições, com o objetivo de prever a capacidade de pagamento de novos empréstimos.

 - application_{train|test}.csv: Esta é a tabela principal, dividida em dois arquivos: Treinamento (com a variável TARGET) e Teste (sem TARGET). Contém dados estáticos de todas as solicitações de empréstimo. Cada linha representa um empréstimo.

 - bureau.csv: Todos os créditos anteriores dos clientes fornecidos por outras instituições financeiras, reportados ao Bureau de Crédito (apenas para clientes com empréstimo na amostra). Cada linha representa um crédito anterior registrado antes da data da aplicação atual.

 - bureau_balance.csv: Saldos mensais dos créditos anteriores no Bureau de Crédito. Cada linha representa um mês de histórico para cada crédito anterior — ou seja, número de empréstimos * número de créditos anteriores * número de meses com histórico disponível.

 - POS_CASH_balance.csv: Saldos mensais dos empréstimos anteriores no ponto de venda (POS) ou empréstimos em dinheiro que o cliente teve com a Home Credit. Uma linha por mês de histórico de cada crédito anterior relacionado ao empréstimo na amostra.

 - credit_card_balance.csv: Saldos mensais dos cartões de crédito anteriores que o cliente tem com a Home Credit. Formato similar ao POS_CASH_balance, mas para cartões de crédito.

 - previous_application.csv: Todas as solicitações anteriores de empréstimos feitas à Home Credit por clientes com empréstimos na amostra atual. Uma linha por solicitação anterior.

 - installments_payments.csv: Histórico de pagamentos de créditos anteriores concedidos pela Home Credit. Inclui: a) uma linha para cada pagamento feito, e b) uma linha para cada parcela não paga. Cada linha representa um pagamento ou uma parcela de um crédito anterior.

 - HomeCredit_columns_description.csv: Arquivo com a descrição das colunas de todos os outros arquivos do conjunto de dados.

 As tabelas podem se relacionar conforme imagem abaixo:

 ![image.png](attachment:image.png)




# PREPARAÇÃO DA BASE DE DADOS - Todos os dados

In [1]:
#Importando Bibliotecas Pandas e Numpy
import pandas as pd
import numpy as np
import os, sys


#Otimizador Giovanni
from sklearnex import patch_sklearn
patch_sklearn()


CSV_PATH = "/home/gi/Desktop/Semestre8/Inteligencia_Computacional/TP1_csvs" # Path Giovanni
# CSV_PATH = "E:/Documentos/CEFET/OneDrive/Documentos/2ECOM067_INTELIGENCIA-COMPUTACIONAL-I_T01/TP1/bases/" # Path Jorge

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [2]:
#Desabilitando os warnings
import warnings
warnings.simplefilter("ignore")

In [ ]:
#Carregando Base de dados principal - Treino
df_train = pd.read_csv(f"{CSV_PATH}/application_train.csv")
df_01 = pd.read_csv(f"{CSV_PATH}/df_01.csv")
df_02 = pd.read_csv(f"{CSV_PATH}/df_02.csv")
df_03 = pd.read_csv(f"{CSV_PATH}/df_03.csv")
df_04 = pd.read_csv(f"{CSV_PATH}/df_04.csv")

y_train = df_train['TARGET']


In [ ]:
df_01.info(max_cols=1000)

# PREPARAÇÃO DA BASE DE DADOS - Amostra

Como as bases de dados original tem muitos valores, e as simulações subsequentes podem demorar muitas horas, realizar-se-a uma amostra aleatória das bases para treinamento e teste.
Para deixar as bases com os mesmos SK_ID_CURR, faz-se primeiramente a amostragem via df_01, e para os demais dataframes selecionamos os valores dos atributos (colunas) do SK_ID_CURR do df_01. Isso faz com que o y_train seja único para os 4 df_x.


In [24]:
def generate_all_xtrain_ytrain(df_01, df_02, df_03, df_04):
    df_01_train = df_01.sample(frac=0.1, random_state=42)  # Seleciona 10% do DataFrame
    df_02_train = df_02.sample(frac=0.1, random_state=42)  # Seleciona 10% do DataFrame
    df_03_train = df_03.sample(frac=0.1, random_state=42)  # Seleciona 10% do DataFrame
    df_04_train = df_04.sample(frac=0.1, random_state=42)  # Seleciona 10% do DataFrame

    #Divisão treino
    X_train_df_01 = df_01_train.drop(columns={"TARGET"})
    y_train_df_01 = df_01_train["TARGET"]
    X_train_df_02 = df_02_train.drop(columns={"TARGET"})
    y_train_df_02 = df_02_train["TARGET"]
    X_train_df_03 = df_03_train.drop(columns={"TARGET"})
    y_train_df_03 = df_03_train["TARGET"]
    X_train_df_04 = df_04_train.drop(columns={"TARGET"})
    y_train_df_04 = df_04_train["TARGET"]
    return  X_train_df_01, y_train_df_01, X_train_df_02, y_train_df_02, X_train_df_03, y_train_df_03, X_train_df_04, y_train_df_04

In [9]:
#Amostra df_01_train
df_01_train = df_01.sample(frac=0.1, random_state=42)  # Seleciona 10% do DataFrame


#Divisão treino
X_train_df_01 = df_01_train.drop(columns={"TARGET"})
y_train_df_01 = df_01_train["TARGET"]

# 1. Treinamento Individual dos Classificadores

In [6]:
#Importanto as Bibliotecas dos classificadores
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.linear_model import LogisticRegression

# Importando biblioteca de clone
from sklearn.base import clone

# Importando stratified k-fold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

### Avaliando o desempenho dos classificadores

In [29]:
import four_dataframes_from1 as fd
df_train = pd.read_csv(f"{CSV_PATH}/application_train.csv")
df_01, df_02, df_03, df_04 = fd.four_dataframes_from_one(df_train, sample=0.1)
y_train = df_01["TARGET"]
x_1 = df_01.drop(columns={"TARGET"})
x_2 = df_02.drop(columns={"TARGET"})
x_3 = df_03.drop(columns={"TARGET"})
x_4 = df_04.drop(columns={"TARGET"})

df_test shape (307511, 122), df_01 shape (30751, 140), df_02 shape (30751, 165), df_03 shape (30751, 165), df_04 shape (30751, 180)


In [30]:
# Confirmando que todos os targets sao os mesmos
if df_01["TARGET"].equals(df_02["TARGET"]) and df_01["TARGET"].equals(df_03["TARGET"]) and df_01["TARGET"].equals(df_04["TARGET"]):
    print("Os dataframes possuem a mesma coluna TARGET")

Os dataframes possuem a mesma coluna TARGET


In [26]:
xgb = XGBClassifier()

# Verificando so com o primeiro df
for train_index, test_index in skf.split(x_1, y_train):
    # Divide os dados de teste
    y_test_fold = y_train.iloc[test_index]
    x_1_test = x_1.iloc[test_index]
    xgb.fit(x_1.iloc[train_index], y_train.iloc[train_index])
    # Predições de classe (0 ou 1)
    xgb_pred_proba = xgb.predict_proba(x_1_test)[:, 1]  # Probabilidade da classe 1
    roc_auc = roc_auc_score(y_test_fold, xgb_pred_proba)
    print(f"Predição 1:  roc_auc = {roc_auc:.4f}")

    # AUC para predições finais
    # roc_auc = roc_auc_score(y_test_fold, pred_final)
    # roc_auc_scores.append(roc_auc)
    
    # Cada modelo usa um X diferente para prever
    

Predição 1:  roc_auc = 0.6977
Predição 1:  roc_auc = 0.7077
Predição 1:  roc_auc = 0.6897
Predição 1:  roc_auc = 0.7231
Predição 1:  roc_auc = 0.6893
Predição 1:  roc_auc = 0.6898
Predição 1:  roc_auc = 0.6961
Predição 1:  roc_auc = 0.7270
Predição 1:  roc_auc = 0.7154
Predição 1:  roc_auc = 0.6874


In [39]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.base import clone
import numpy as np

def avaliar_modelo_meta_individual(model, X_1, X_2, X_3, X_4, y_train, nome):
    roc_auc_scores = []

    for train_index, test_index in skf.split(X_1, y_train):
        # Divide os dados de teste
        y_test_fold = y_train.iloc[test_index]

        # Dados de teste para cada modelo
        X1_test = X_1.iloc[test_index]
        X2_test = X_2.iloc[test_index]
        X3_test = X_3.iloc[test_index]
        X4_test = X_4.iloc[test_index]

        # Clona e treina os modelos em seus respectivos datasets
        m1 = clone(model).fit(X_1.iloc[train_index], y_train.iloc[train_index])
        m2 = clone(model).fit(X_2.iloc[train_index], y_train.iloc[train_index])
        m3 = clone(model).fit(X_3.iloc[train_index], y_train.iloc[train_index])
        m4 = clone(model).fit(X_4.iloc[train_index], y_train.iloc[train_index])

        # Obtém as probabilidades da classe positiva
        p1 = (m1.predict_proba(X1_test)[:, 1] >= 0.5).astype(int)
        p2 = (m2.predict_proba(X2_test)[:, 1] >= 0.5).astype(int)
        p3 = (m3.predict_proba(X3_test)[:, 1] >= 0.5).astype(int)
        p4 = (m4.predict_proba(X4_test)[:, 1] >= 0.5).astype(int)

        # Avaliação individual opcional
        print(f"Predição 1:  accuracy_auc = {accuracy_score(y_test_fold, p1):.4f}")
        print(f"Predição 2:  accuracy_auc = {accuracy_score(y_test_fold, p2):.4f}")
        print(f"Predição 3:  accuracy_auc = {accuracy_score(y_test_fold, p3):.4f}")
        print(f"Predição 4:  accuracy_auc = {accuracy_score(y_test_fold, p4):.4f}")   

        # Ensemble: 
        preds = np.column_stack([p1, p2, p3, p4])
        print(f"{preds}")
        pred_final = (preds.sum(axis=1) >= 2).astype(int)

        # Avaliação do ensemble
        roc_auc = roc_auc_score(y_test_fold, pred_final)
        roc_auc_scores.append(roc_auc)
        
        accuracy = accuracy_score(y_test_fold, pred_final)
        f1 = f1_score(y_test_fold, pred_final)
    print(f"Predição final:  accuracy_auc = {accuracy:.4f}")
    print(f"Predição final:  f1_auc = {f1:.4f}")
    #print(f"{nome}: roc_auc_mean = {np.mean(roc_auc_scores):.4f}, desvio padrão = {np.std(roc_auc_scores):.4f}")
    return np.mean(roc_auc_scores)


In [40]:
avaliar_modelo_meta_individual(XGBClassifier(), x_1, x_2, x_3, x_4, y_train, "XGBoost")

Predição 1:  accuracy_auc = 0.9125
Predição 2:  accuracy_auc = 0.9158
Predição 3:  accuracy_auc = 0.9161
Predição 4:  accuracy_auc = 0.9161
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 ...
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Predição 1:  accuracy_auc = 0.9138
Predição 2:  accuracy_auc = 0.9148
Predição 3:  accuracy_auc = 0.9164
Predição 4:  accuracy_auc = 0.9141
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 ...
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Predição 1:  accuracy_auc = 0.9148
Predição 2:  accuracy_auc = 0.9128
Predição 3:  accuracy_auc = 0.9145
Predição 4:  accuracy_auc = 0.9099
[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]
 ...
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Predição 1:  accuracy_auc = 0.9187
Predição 2:  accuracy_auc = 0.9138
Predição 3:  accuracy_auc = 0.9148
Predição 4:  accuracy_auc = 0.9145
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 ...
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
Predição 1:  accuracy_auc = 0.9161
Predição 2:  accuracy_auc = 0.9138
Predição 3:  accuracy_auc = 0.9161
Predição 4:  accuracy_auc = 0.9148
[[0 0 0 0]
 

np.float64(0.5226711999245983)

In [ ]:
from sklearn.base import clone
xgb = XGBClassifier(
    random_state=42,
)
lgbm = LGBMClassifier(
    random_state=42,
)
catboosta = CatBoostClassifier(
    random_state=42,
    verbose=0,  # Desativa a saída de log do CatBoost
)
X_train_df_01, y_train_df_01, X_train_df_02, y_train_df_02, X_train_df_03, y_train_df_03, X_train_df_04, y_train_df_04 = generate_all_xtrain_ytrain(df_01, df_02, df_03, df_04)

X_trains = [X_train_df_01, X_train_df_02, X_train_df_03, X_train_df_04]
y_trains = [y_train_df_01, y_train_df_02, y_train_df_03, y_train_df_04]

xgb1 = clone(xgb).fit(X_train_df_01, y_train_df_01)
xgb2 = clone(xgb).fit(X_train_df_02, y_train_df_02)
xgb3 = clone(xgb).fit(X_train_df_03, y_train_df_03)
xgb4 = clone(xgb).fit(X_train_df_04, y_train_df_04)

for i, (X_train, y_train) in enumerate(zip(X_trains, y_trains)):
    print(f"Modelo {i+1}")
    # Avaliando o modelo XGBClassifier
    avaliar_modelo_individual(xgb, X_train, y_train, f"XGBClassifier{i+1}")
    

Modelo 1
XGBClassifier1: roc_auc = 0.7044, desvio padrão = 0.0000
Modelo 2
XGBClassifier2: roc_auc = 0.7023, desvio padrão = 0.0000
Modelo 3
XGBClassifier3: roc_auc = 0.7057, desvio padrão = 0.0000
Modelo 4
XGBClassifier4: roc_auc = 0.7036, desvio padrão = 0.0000


### Criando os modelos individuais

In [29]:
xgb1 = clone(xgb).fit(X_train_df_01, y_train_df_01)
xgb2 = clone(xgb).fit(X_train_df_02, y_train_df_02)
xgb3 = clone(xgb).fit(X_train_df_03, y_train_df_03)
xgb4 = clone(xgb).fit(X_train_df_04, y_train_df_04)

# 2. Combinação dos modelos

Pegou-se os modelos com melhores resultados dos dataframes df_01, df_02, df_03 e df_04 e aplicou a estratégia de combinação deles

In [42]:
#Importando Bibliotecas pertinentes
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

# Suponha que X_meta e y_meta sejam os dados que TODOS os modelos podem prever
# Pode ser um subconjunto comum ou seu conjunto de teste

def avaliar_modelo_final(X1, X2, X3, X4, Y, nome):
    roc_auc_scores = []
    for train_index, test_index in skf.split(X_train, y_train):
        X_test_fold = X.iloc[test_index]
        y_test_fold = Y.iloc[test_index]
         # Predições de classe (0 ou 1)
        pred1 = xgb1.predict(X_test_fold)
        pred2 = xgb2.predict(X_test_fold)
        pred3 = xgb3.predict(X_test_fold)
        pred4 = xgb4.predict(X_test_fold)
         # Stack horizontalmente: shape (n amostras, 4 modelos)
        preds = np.column_stack([pred1, pred2, pred3, pred4])
        pred_final = (preds.sum(axis=1) >= 2).astype(int)
        # Calcule a pontuação ROC AUC
        roc_auc = roc_auc_score(y_test_fold, pred_final)
        # ROC AUC
        roc_auc = roc_auc_score(y_test_fold, pred_final)
        roc_auc_scores.append(roc_auc)
    print(f"{nome}: ROC AUC mean = {np.mean(roc_auc_scores):.4f}, desvio padrão = {np.std(roc_auc_scores):.4f}")
    return np.mean(roc_auc_scores)


In [ ]:
avaliar_modelo_final()

In [47]:
#Teste com o dataframe de Teste Global

#Realizando o treinamento com base no X1
estimador_df.fit(X1, y)
#Realizando a predição com base no X1_teste
preds_1_teste = estimador_df.predict(X1_test)

#Realizando o treinamento com base no X2
estimador_df.fit(X2, y)
#Realizando a predição com base no X2_teste
preds_2_teste = estimador_df.predict(X2_test)

#Realizando o treinamento com base no X3
estimador_df.fit(X3, y)
#Realizando a predição com base no X3_teste
preds_3_teste = estimador_df.predict(X3_test)

#Realizando o treinamento com base no X4
estimador_df.fit(X4, y)
#Realizando a predição com base no X4_teste
preds_4_teste = estimador_df.predict(X4_test)

soma = preds_1_teste + preds_2_teste + preds_3_teste + preds_4_teste

preds = [1 if s > 2 else 0 for s in soma]

#Geração do Data Frame das predições com o Teste Global
df_kaggle = df_test.copy()
df_kaggle["TARGET"] = preds

# Salvar os CSVs
df_kaggle[["SK_ID_CURR", "TARGET"]].to_csv("df_kaggle.csv", index=False)

#RESULTADOS

![image.png](attachment:image.png)

Como informado anteriormente, devido a grande quantidade de dados e limitações estruturais e de capacidade do notebook não foi possível criar uma base única com as devidas mesclagens. Como alternativa, montou-se 4 Data Frames (df_01 a df_04) com join de bases diferentes, mas tento uma base principal. Mas como a base inicial ainda continuava grande, optou-se por amostrar em 20% a base original, aleatoriamente. Cada Data Frame, gerado aleatoriamente, foi utilizado para treinar modelos dos classificadores da prática (XGBOOST, LIGHTGBM e CATBOOST), tendo como métrica de avaliação o ROC-UAC.

Para o df_01 obteve-se como resultados:

![image.png](attachment:image.png)


Para o df_02 obteve-se como resultados:

![image-2.png](attachment:image-2.png)


Para o df_03 obteve-se como resultados:

![image-3.png](attachment:image-3.png)


Para o df_04 obteve-se como resultados:

![image-4.png](attachment:image-4.png)


Observou-se que os modelos que sairam melhor foram o XGBOOST, LIGHTGBM, ambos com resultados próximos. Já o CATBOOST foi o de pior resultado em todos os dataframes.

No intuito de agregar os diferentes modelos para uma melhor resposta, utilizou-se a estratégia a combinação com VotingClassifier (Hard e Soft) e StackingClassifier. A combinação consiste na combinação de modelos, gerando um novo modelo, e com este novo modelo há o treinamento de uma base. Como nossa base não foi única, e sim fragmentada nos 4 data frames, optou-se por escolher o df_01 como base para X e y na combinação. A escolha foi feita por entendermos que o df_01 trazia mais informações, no entanto, ficamos com dúvida se está é uma boa e ideal escolha.

Para o VotingClassifier (Hard e Soft) obteve-se os seguintes resultados:

![image-5.png](attachment:image-5.png)

![image-7.png](attachment:image-7.png)

Para o StackingClassifier obteve-se os seguinte resultado

![image-6.png](attachment:image-6.png)

Para ambos os modelos gerados através da combinação houve a predição com o dataframe de test do Kaggle. Gerou-se 2 arquivos . CSV que foram submetidos no Kaggle, obtendo-se os seguintes resultados:

![image-8.png](attachment:image-8.png)

Em comparação com as simulações individuais  observou-se que houve a piora pela combinação do modelos. No entanto, os resultados da combinação foram mais próximos dos resultados após submissão no Kaggle. Isto sugere que os modelos individuais estão mais overfitados. No entanto, tem-se que ponderar que somente 20% de dados ( aproximadamente 66 mil) foram utilizados.


utilizar label encoder